<a href="https://colab.research.google.com/github/ChacoGolden/Stat-Data-Science/blob/main/Panel_reg_edu_int_infl_hh_wm_age_urb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install linearmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.5/94.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
from linearmodels.panel import PanelOLS
from statsmodels.tools import add_constant

In [3]:
# Load the CSV file
data = pd.read_csv('https://raw.githubusercontent.com/ChacoGolden/Stat-Data-Science/refs/heads/main/Panel_Data_Inc_Edu_Infl_Wm_Hh_Age_Urb.csv')
# Convert 'Waste' from tons to kilograms
data['Waste'] = data['Waste'] * 1000
# Inspect the first few rows of the data
print(data.head())

    Country  Year       Waste   Income  Education  Inflation  Women  \
0   Belgium  2014  484.672171  21705.0       32.6        0.5   62.9   
1  Bulgaria  2014  376.962939   3311.0       23.6       -1.6   61.5   
2   Czechia  2014  310.164578   7622.0       19.1        0.4   64.7   
3   Denmark  2014  636.289226  27861.0       29.4        0.4   71.0   
4   Germany  2014  456.714036  19733.0       23.2        0.8   72.1   

   Household   Age    Urban  
0        2.3  41.3  0.97833  
1        2.6  43.7  0.73651  
2        2.4  40.8  0.73384  
3        2.0  41.3  0.87410  
4        2.0  45.6  0.77190  


In [4]:
# Convert 'Country' and 'Year' to categorical variables (if not already)
data['Country'] = data['Country'].astype('category')
data['Year'] = data['Year'].astype('category')

# Check for missing values
print(data.isnull().sum())
data = data[data['Country'] != 'Türkiye']
print(data.head())
num_countries = data['Country'].nunique()
print(f"Number of different countries: {num_countries}")
print(data.isnull().sum())

Country      0
Year         0
Waste        0
Income       0
Education    0
Inflation    0
Women        0
Household    0
Age          1
Urban        0
dtype: int64
    Country  Year       Waste   Income  Education  Inflation  Women  \
0   Belgium  2014  484.672171  21705.0       32.6        0.5   62.9   
1  Bulgaria  2014  376.962939   3311.0       23.6       -1.6   61.5   
2   Czechia  2014  310.164578   7622.0       19.1        0.4   64.7   
3   Denmark  2014  636.289226  27861.0       29.4        0.4   71.0   
4   Germany  2014  456.714036  19733.0       23.2        0.8   72.1   

   Household   Age    Urban  
0        2.3  41.3  0.97833  
1        2.6  43.7  0.73651  
2        2.4  40.8  0.73384  
3        2.0  41.3  0.87410  
4        2.0  45.6  0.77190  
Number of different countries: 25
Country      0
Year         0
Waste        0
Income       0
Education    0
Inflation    0
Women        0
Household    0
Age          0
Urban        0
dtype: int64


Il manque les data pour l'âge moyen en Turquie

In [5]:
data['Year'] = data['Year'].astype(int)  # Or float if years are not integers

In [6]:
# Set 'Country' and 'Year' as index to create a MultiIndex
data = data.set_index(['Country', 'Year'])

In [11]:
# Define the dependent variable (Y) and independent variables (X)
Y = data['Waste']
X = data[['Income', 'Education', 'Women', 'Household','Urban', 'Age']]

# Add a constant term to the independent variables
X = add_constant(X)

In [10]:
# Run the fixed effects model
model = PanelOLS(dependent=Y, exog=X, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

# Display the results
print(results.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                  Waste   R-squared:                        0.1060
Estimator:                   PanelOLS   R-squared (Between):             -0.6008
No. Observations:                 225   R-squared (Within):               0.2174
Date:                Wed, Dec 04 2024   R-squared (Overall):             -0.4345
Time:                        16:02:32   Log-likelihood                   -1116.3
Cov. Estimator:                Robust                                           
                                        F-statistic:                      4.4339
Entities:                          25   P-value                           0.0008
Avg Obs:                       9.0000   Distribution:                   F(5,187)
Min Obs:                       9.0000                                           
Max Obs:                       9.0000   F-statistic (robust):             5.0827
                            

/usr/local/lib/python3.10/dist-packages/linearmodels/panel/data.py:590: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_mu = self._frame.groupby(level=level).transform("mean")
/usr/local/lib/python3.10/dist-packages/linearmodels/panel/data.py:590: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_mu = self._frame.groupby(level=level).transform("mean")
/usr/local/lib/python3.10/dist-packages/linearmodels/panel/data.py:590: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to

In [12]:
# Create the interaction term
data['Income_Education'] = data['Income'] * data['Education']
data['Income_Age'] = data['Income'] * data['Age']

In [13]:
# Define the dependent variable (Y) and independent variables (X)
Y = data['Waste']
X = data[['Income', 'Education', 'Inflation', 'Income_Education', 'Women', 'Household', 'Age', 'Urban', 'Income_Age']]

# Add a constant term to the independent variables
X = add_constant(X)

# Run the fixed effects model with interaction term
model = PanelOLS(dependent=Y, exog=X, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

# Display the results
print(results.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                  Waste   R-squared:                        0.2881
Estimator:                   PanelOLS   R-squared (Between):             -1.2960
No. Observations:                 225   R-squared (Within):              -0.0439
Date:                Wed, Dec 04 2024   R-squared (Overall):             -1.0415
Time:                        16:03:04   Log-likelihood                   -1090.7
Cov. Estimator:                Robust                                           
                                        F-statistic:                      8.2277
Entities:                          25   P-value                           0.0000
Avg Obs:                       9.0000   Distribution:                   F(9,183)
Min Obs:                       9.0000                                           
Max Obs:                       9.0000   F-statistic (robust):             12.042
                            

/usr/local/lib/python3.10/dist-packages/linearmodels/panel/data.py:590: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_mu = self._frame.groupby(level=level).transform("mean")
/usr/local/lib/python3.10/dist-packages/linearmodels/panel/data.py:590: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_mu = self._frame.groupby(level=level).transform("mean")
/usr/local/lib/python3.10/dist-packages/linearmodels/panel/data.py:590: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to

The interaction is negative and significant, meaning the relationship between Income and Waste weakens as Education increases.

In [ ]:
import numpy as np

# Transform 'Income' to its logarithm
data['Log_Income'] = np.log(data['Income'])


In [ ]:
# Update the interaction term to reflect the log transformation
data['Log_Income_Education'] = data['Log_Income'] * data['Education']
data['Log_Income_Age'] = data['Log_Income'] * data['Age']

X['Log_Income_Education'] = data['Log_Income_Education']

# Define the dependent variable (Y) and independent variables (X)
Y = data['Waste']
X = data[['Log_Income', 'Education', 'Inflation', 'Log_Income_Education', 'Women', 'Household', 'Urban', 'Log_Income_Age', 'Age']]



# Add a constant term to the independent variables
X = add_constant(X)

# Run the fixed effects model
model = PanelOLS(dependent=Y, exog=X, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

# Display the results
print(results.summary)


                          PanelOLS Estimation Summary                           
Dep. Variable:                  Waste   R-squared:                        0.2964
Estimator:                   PanelOLS   R-squared (Between):             -1.1911
No. Observations:                 225   R-squared (Within):              -0.1627
Date:                Wed, Dec 04 2024   R-squared (Overall):             -0.9821
Time:                        13:38:35   Log-likelihood                   -1089.4
Cov. Estimator:                Robust                                           
                                        F-statistic:                      8.5666
Entities:                          25   P-value                           0.0000
Avg Obs:                       9.0000   Distribution:                   F(9,183)
Min Obs:                       9.0000                                           
Max Obs:                       9.0000   F-statistic (robust):             12.303
                            

/usr/local/lib/python3.10/dist-packages/linearmodels/panel/data.py:590: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_mu = self._frame.groupby(level=level).transform("mean")
/usr/local/lib/python3.10/dist-packages/linearmodels/panel/data.py:590: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_mu = self._frame.groupby(level=level).transform("mean")
/usr/local/lib/python3.10/dist-packages/linearmodels/panel/data.py:590: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to

In [ ]:
# Update the interaction term to reflect the log transformation
data['Log_Income_Education'] = data['Log_Income'] * data['Education']
data['Log_Income_Age'] = data['Log_Income'] * data['Age']

X['Log_Income_Education'] = data['Log_Income_Education']

# Define the dependent variable (Y) and independent variables (X)
Y = data['Waste']
X = data[['Log_Income', 'Education', 'Inflation', 'Log_Income_Education', 'Household', 'Log_Income_Age', 'Age']]



# Add a constant term to the independent variables
X = add_constant(X)

# Run the fixed effects model
model = PanelOLS(dependent=Y, exog=X, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

# Display the results
print(results.summary)


                          PanelOLS Estimation Summary                           
Dep. Variable:                  Waste   R-squared:                        0.2951
Estimator:                   PanelOLS   R-squared (Between):             -0.5406
No. Observations:                 225   R-squared (Within):              -0.1563
Date:                Wed, Dec 04 2024   R-squared (Overall):             -0.4625
Time:                        14:10:18   Log-likelihood                   -1089.6
Cov. Estimator:                Robust                                           
                                        F-statistic:                      11.062
Entities:                          25   P-value                           0.0000
Avg Obs:                       9.0000   Distribution:                   F(7,185)
Min Obs:                       9.0000                                           
Max Obs:                       9.0000   F-statistic (robust):             13.531
                            

/usr/local/lib/python3.10/dist-packages/linearmodels/panel/data.py:590: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_mu = self._frame.groupby(level=level).transform("mean")
/usr/local/lib/python3.10/dist-packages/linearmodels/panel/data.py:590: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_mu = self._frame.groupby(level=level).transform("mean")
/usr/local/lib/python3.10/dist-packages/linearmodels/panel/data.py:590: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to

In [ ]:
# Update the interaction term to reflect the log transformation
data['Log_Income_Education'] = data['Log_Income'] * data['Education']
data['Log_Income_Age'] = data['Log_Income'] * data['Age']

X['Log_Income_Education'] = data['Log_Income_Education']

# Define the dependent variable (Y) and independent variables (X)
Y = data['Waste']
X = data[['Log_Income', 'Education', 'Inflation', 'Log_Income_Education', 'Women', 'Household','Urban']]



# Add a constant term to the independent variables
X = add_constant(X)

# Run the fixed effects model
model = PanelOLS(dependent=Y, exog=X, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

# Display the results
print(results.summary)


                          PanelOLS Estimation Summary                           
Dep. Variable:                  Waste   R-squared:                        0.2312
Estimator:                   PanelOLS   R-squared (Between):             -0.8072
No. Observations:                 225   R-squared (Within):              -0.2758
Date:                Wed, Dec 04 2024   R-squared (Overall):             -0.6992
Time:                        14:12:24   Log-likelihood                   -1099.4
Cov. Estimator:                Robust                                           
                                        F-statistic:                      7.9496
Entities:                          25   P-value                           0.0000
Avg Obs:                       9.0000   Distribution:                   F(7,185)
Min Obs:                       9.0000                                           
Max Obs:                       9.0000   F-statistic (robust):             9.9427
                            

/usr/local/lib/python3.10/dist-packages/linearmodels/panel/data.py:590: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_mu = self._frame.groupby(level=level).transform("mean")
/usr/local/lib/python3.10/dist-packages/linearmodels/panel/data.py:590: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_mu = self._frame.groupby(level=level).transform("mean")
/usr/local/lib/python3.10/dist-packages/linearmodels/panel/data.py:590: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to

In [ ]:
# Log-transform 'Income' and 'Waste'
data['Log_Waste'] = np.log(data['Waste'])
data['Log_Income'] = np.log(data['Income'])
# Update the interaction term to use log-transformed income
data['Log_Income_Education'] = data['Log_Income'] * data['Education']


In [ ]:
# Define the dependent variable (Y) and independent variables (X)
Y = data['Log_Waste']
X = data[['Log_Income', 'Education', 'Inflation', 'Log_Income_Education']]

# Add a constant term
X = add_constant(X)

# Run the fixed effects model
model = PanelOLS(dependent=Y, exog=X, entity_effects=True, time_effects=True)
results = model.fit()

# Display the results
print(results.summary)

/usr/local/lib/python3.10/dist-packages/linearmodels/panel/data.py:590: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_mu = self._frame.groupby(level=level).transform("mean")
/usr/local/lib/python3.10/dist-packages/linearmodels/panel/data.py:590: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_mu = self._frame.groupby(level=level).transform("mean")
/usr/local/lib/python3.10/dist-packages/linearmodels/panel/data.py:590: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to

                          PanelOLS Estimation Summary                           
Dep. Variable:              Log_Waste   R-squared:                        0.2013
Estimator:                   PanelOLS   R-squared (Between):             -0.1948
No. Observations:                 225   R-squared (Within):              -0.0318
Date:                Wed, Dec 04 2024   R-squared (Overall):             -0.1631
Time:                        12:49:33   Log-likelihood                    238.64
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      11.844
Entities:                          25   P-value                           0.0000
Avg Obs:                       9.0000   Distribution:                   F(4,188)
Min Obs:                       9.0000                                           
Max Obs:                       9.0000   F-statistic (robust):             11.844
                            

In [ ]:
!pip install statsmodels

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
X = data[['Income', 'Education', 'Inflation', 'Women', 'Household', 'Age', 'Urban']]
X = sm.add_constant(X)  # Add a constant term (required for VIF calculation)

In [ ]:
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data)

    Variable          VIF
0      const  2505.899470
1     Income     2.577251
2  Education     2.569911
3  Inflation     1.213238
4      Women     2.459303
5  Household     3.635550
6        Age     2.101014
7      Urban     1.895764
